In [ ]:
import numpy as np
import pandas as pd

# from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression, Lasso


1. Использовать dataset MovieLens 
https://grouplens.org/datasets/movielens/latest/

описание датафреймов:
df_rat - дф с рейтингом
df_tags - дф с тегами
df_movies - фильмы = жанры + название

movie_genres - жанры
df_tfidf_g - tfidf по жанрам
df_movie_g = df_tfidf_g  + df_movies(без жанра)

rat_with_tags = df_tags + df_rat
df_tfidf_t - tfidf по тегам

rat_dfidf_t = rat_with_tags(без тегов) + df_tfidf_t

m_dg = df_tfidf_g + rat_dfidf_t жанры + теги + рейтинг
-по последней строим предсказание рейтинга от жанров и тегов


plus_mean_film - добавление столбца среднего по фильму к m_dg

plus_mean_film_and_user - к столбцу среднего по фильму + столбец среднего по пользователю

In [ ]:
df_rat = pd.read_csv('/content/ratings.csv')
df_rat.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
len(df_rat)

100004

In [ ]:
df_tags = pd.read_csv('/content/tags.csv')
df_tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [ ]:
len(df_tags['userId'].value_counts())

61

In [ ]:
len(df_tags)

1296

In [ ]:
df_movies = pd.read_csv('/content/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
len(df_movies.genres.values)

9125

2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:

TF-IDF на тегах и жанрах
Средние оценки (+ median, variance, etc.) пользователя и фильма

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.neighbors import NearestNeighbors

In [ ]:
def change_string(s):
    return s.replace(' ', '').replace('-', '')
movie_genres = df_movies['genres'].apply(change_string)
movie_genres

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9120                        Adventure|Drama|Romance
9121                 Action|Adventure|Fantasy|SciFi
9122                                    Documentary
9123                                         Comedy
9124                                    Documentary
Name: genres, Length: 9125, dtype: object

In [ ]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9125x20 sparse matrix of type '<class 'numpy.float64'>'
	with 20340 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf_g = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [ ]:
df_tfidf_g['movieId'] = df_movies['movieId']
df_tfidf_g.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,movieId
0,0.0,0.410433,0.531527,0.496423,0.266469,0.0,0.0,0.000000,0.481230,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
1,0.0,0.510466,0.000000,0.617414,0.000000,0.0,0.0,0.000000,0.598519,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2
2,0.0,0.000000,0.000000,0.000000,0.586999,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.809588,0.0,0.0,0.0,0.0,3
3,0.0,0.000000,0.000000,0.000000,0.523605,0.0,0.0,0.452029,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.722155,0.0,0.0,0.0,0.0,4
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5


In [ ]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# len(df_tfidf_g)== len(df_movies)
# tfidf_g + movie([title])
df_movie_g = df_tfidf_g.merge(df_movies.drop(['genres'],axis=1))
df_movie_g.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,movieId,title
0,0.0,0.410433,0.531527,0.496423,0.266469,0.0,0.0,0.000000,0.481230,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1,Toy Story (1995)
1,0.0,0.510466,0.000000,0.617414,0.000000,0.0,0.0,0.000000,0.598519,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2,Jumanji (1995)
2,0.0,0.000000,0.000000,0.000000,0.586999,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.809588,0.0,0.0,0.0,0.0,3,Grumpier Old Men (1995)
3,0.0,0.000000,0.000000,0.000000,0.523605,0.0,0.0,0.452029,0.000000,0.0,...,0.0,0.0,0.0,0.722155,0.0,0.0,0.0,0.0,4,Waiting to Exhale (1995)
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5,Father of the Bride Part II (1995)


In [ ]:
len(df_movie_g)

9125

In [ ]:
from tqdm.notebook import tqdm

In [ ]:

# пока не дошло до groupby объединю рейтинг и теги по 2ум столбцам
# тэгов меньше, есть по несколько тегов одномц фильму-одного порльзователя
len(df_movies), len(df_tags), len(df_rat)

(9125, 1296, 100004)

In [ ]:
rat_with_tags = df_tags.merge(df_rat, on=['movieId','userId'])
rat_with_tags.columns = rat_with_tags.columns.str.strip()
rat_with_tags.head()

,userId,movieId,tag,timestamp_x,rating,timestamp_y
0,15,339,sandra 'boring' bullock,1138537770,2.5,1122576622
1,15,1955,dentist,1193435061,4.5,1338698424
2,15,34162,forgettable,1141391765,1.0,1141391741
3,15,35957,short,1141391873,2.5,1141391844
4,15,37729,dull story,1141391806,2.0,1141391812


In [ ]:
len(rat_with_tags)

939

In [ ]:
rat_with_tags.dropna(inplace=True)
len(rat_with_tags)

939

In [ ]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '')

tag_strings = []
movies = []
user_id = []

for movie, group in tqdm(rat_with_tags.groupby('movieId')):
    tag_strin = ' '.join([change_string(s) for s in group.tag.values])
    for user in group.userId.values:
      tag_strings.append(tag_strin)
      movies.append(movie)
      user_id.append(user)
    # print(movie)
    # print(group.userId.values)
    # ну и толку?( есть не по одному юзеру в группе
    # user_id.append(group.userId.values)
    # print(group['userId'])
    # break

  0%|          | 0/445 [00:00<?, ?it/s]

In [ ]:
rat_with_tags[rat_with_tags['movieId'] ==  47]

,userId,movieId,tag,timestamp_x,rating,timestamp_y
278,364,47,biblical,1444534976,0.5,1444534953
279,364,47,crime,1444534982,0.5,1444534953
280,364,47,dark,1444534994,0.5,1444534953
281,364,47,disturbing,1444534971,0.5,1444534953
282,364,47,greed,1444534998,0.5,1444534953
283,364,47,horror,1444534981,0.5,1444534953
284,364,47,serial killer,1444534961,0.5,1444534953
285,364,47,violent,1444534985,0.5,1444534953
622,501,47,psychology,1292956276,5.0,1283137699
623,501,47,twist ending,1292956362,5.0,1283137699


In [ ]:
len(movies) == len(user_id)

True

In [ ]:
tfidf_tag = TfidfVectorizer()
X_train_tfidf_tag = tfidf_tag.fit_transform(tag_strings)
X_train_tfidf_tag

<939x504 sparse matrix of type '<class 'numpy.float64'>'
	with 4176 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf_t = pd.DataFrame(X_train_tfidf_tag.toarray(), columns=tfidf_tag.get_feature_names_out())
df_tfidf_t['movieId'] = movies
df_tfidf_t['userId'] = user_id
len(rat_with_tags), len(rat_with_tags)

(939, 939)

In [ ]:
# df_tfidf_t.columns.values,rat_with_tags.columns.values

In [ ]:
# len(df_tfidf_t), len(rat_with_tags) == (939, 939)
# тегов типо меньше рейтинга
# а строк тегов, по кол-ву фильмов, за которые были оставлены рейтинги-теги

rat_dfidf_t = pd.merge(rat_with_tags.drop(['tag'], axis = 1), df_tfidf_t,how='left', on=['movieId', 'userId'], suffixes=['_rat', '_tfidf']) 

# rat_dfidf_t = rat_with_tags.drop(['tag'], axis = 1).join(df_tfidf_t, on=['movieId','userId'])
rat_dfidf_t.head()

,userId,movieId,timestamp_x,rating,timestamp_y,1940,80,abigailbreslin,action,activist,...,welldone,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel
0,15,339,1138537770,2.5,1122576622,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,1955,1193435061,4.5,1338698424,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,34162,1141391765,1.0,1141391741,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,35957,1141391873,2.5,1141391844,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,37729,1141391806,2.0,1141391812,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
len(rat_dfidf_t),len(df_tfidf_t), len(rat_with_tags)
# и почему? ( почему-то появились одинаковые строки, придётся удалить дубликаты. но причина не ясна

(3251, 939, 939)

In [ ]:
rat_dfidf_t[rat_dfidf_t['userId'] == 15]

,userId,movieId,timestamp_x,rating,timestamp_y,1940,80,abigailbreslin,action,activist,...,welldone,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel
0,15,339,1138537770,2.5,1122576622,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,1955,1193435061,4.5,1338698424,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,34162,1141391765,1.0,1141391741,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,35957,1141391873,2.5,1141391844,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,37729,1141391806,2.0,1141391812,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,15,45950,1169616291,3.0,1169616283,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,15,100365,1425876220,2.5,1361831156,0.0,0.0,0.0,0.0,0.604517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,15,100365,1425876220,2.5,1361831156,0.0,0.0,0.0,0.0,0.604517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,15,100365,1425876220,2.5,1361831156,0.0,0.0,0.0,0.0,0.604517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,15,100365,1425876220,2.5,1361831156,0.0,0.0,0.0,0.0,0.604517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rat_dfidf_t.drop_duplicates(inplace=True)
len(rat_dfidf_t)

892

In [ ]:
rat_dfidf_t[rat_dfidf_t['userId'] == 15]

,userId,movieId,timestamp_x,rating,timestamp_y,1940,80,abigailbreslin,action,activist,...,welldone,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel
0,15,339,1138537770,2.5,1122576622,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15,1955,1193435061,4.5,1338698424,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,34162,1141391765,1.0,1141391741,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,35957,1141391873,2.5,1141391844,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,37729,1141391806,2.0,1141391812,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,15,45950,1169616291,3.0,1169616283,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,15,100365,1425876220,2.5,1361831156,0.0,0.0,0.0,0.0,0.604517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_tfidf_g.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western,movieId
0,0.0,0.410433,0.531527,0.496423,0.266469,0.0,0.0,0.000000,0.481230,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
1,0.0,0.510466,0.000000,0.617414,0.000000,0.0,0.0,0.000000,0.598519,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,2
2,0.0,0.000000,0.000000,0.000000,0.586999,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.809588,0.0,0.0,0.0,0.0,3
3,0.0,0.000000,0.000000,0.000000,0.523605,0.0,0.0,0.452029,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.722155,0.0,0.0,0.0,0.0,4
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5


In [ ]:
df_tfidf_g.shape,rat_dfidf_t.shape

((9125, 21), (892, 509))

In [ ]:
# у каждого тега есть кино, наоборот не обязательно
m_dg = df_tfidf_g.merge(rat_dfidf_t, how = 'right',on = 'movieId') #;жанры + теги
m_dg.head()

,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,documentary_x,drama_x,fantasy_x,filmnoir,...,welldone,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel
0,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.52611,0.0,0.263753,0.0,0.0,0.0,0.476326,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
m_dg.shape # -1 общий столбец== movieID

(892, 529)

In [ ]:
m_dg[m_dg['movieId']==47][['movieId','mystery','thriller_x','dark','pixar','biblical']]

,movieId,mystery,thriller_x,dark,pixar,biblical
251,47,0.820335,0.571883,0.309801,0.0,0.351524
252,47,0.820335,0.571883,0.309801,0.0,0.351524
253,47,0.820335,0.571883,0.309801,0.0,0.351524
254,47,0.820335,0.571883,0.309801,0.0,0.351524
255,47,0.820335,0.571883,0.309801,0.0,0.351524
256,47,0.820335,0.571883,0.309801,0.0,0.351524
257,47,0.820335,0.571883,0.309801,0.0,0.351524
258,47,0.820335,0.571883,0.309801,0.0,0.351524
581,47,0.820335,0.571883,0.309801,0.0,0.351524
582,47,0.820335,0.571883,0.309801,0.0,0.351524


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(m_dg.drop(['rating'], axis=1), m_dg['rating'], test_size=0.3, random_state=21)

In [ ]:
X_test

,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,documentary_x,drama_x,fantasy_x,filmnoir,...,welldone,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel
687,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287,0.0,0.000000,0.0,0.0,0.402792,0.0,0.0,0.000000,0.727424,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.365634,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
806,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.0,0.759046,0.0,0.0,0.492801,0.0,0.0,0.425436,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
664,0.0,0.000000,0.0,0.0,0.523605,0.0,0.0,0.452029,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652,0.0,0.000000,0.0,0.0,0.523605,0.0,0.0,0.452029,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


3. Оценить RMSE на тестовой выборке

In [ ]:
from sklearn.metrics import mean_squared_error
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
y_pred = model_linear.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_squared_error(y_train, model_linear.predict(X_train))))
# ошибаемся на 1.1 бал на тесте и на0.3 на тренировочной - похоже на переобучение
# попробую другую регрессию

1.1298599892968402
0.31163835921766037


In [ ]:

clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_squared_error(y_train, clf.predict(X_train))))
# тут немного больше ошибаемся на тесте на (0.06 бала), но меньше похоже на переобучение

1.191516116759153
1.1048475448163386


In [ ]:
mean_film = df_rat.groupby('movieId')['rating'].mean()
mean_film.head()

movieId
1    3.872470
2    3.401869
3    3.161017
4    2.384615
5    3.267857
Name: rating, dtype: float64

In [ ]:
# добавим среднее по фильмам
m_dg.shape, mean_film.shape

((892, 529), (9066,))

In [ ]:
plus_mean_film = m_dg.merge(mean_film, on=['movieId'])
plus_mean_film.head()

,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,documentary_x,drama_x,fantasy_x,filmnoir,...,whimsical,whodoneit,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel,rating_y
0,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.440594
1,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.838710
2,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.547619
3,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.500000
4,0.0,0.0,0.52611,0.0,0.263753,0.0,0.0,0.0,0.476326,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.738095


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(plus_mean_film.drop(['rating_y'], axis=1), plus_mean_film['rating_y'], test_size=0.3, random_state=21)

clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_squared_error(y_train, clf.predict(X_train))))
# точность выросла больше чем на пол бала!

0.4716276299358102
0.44063693882126515


In [ ]:
mean_user = df_rat.groupby('userId').mean()

In [ ]:
# добавим к последнему среднее по пользователям
plus_mean_film_and_user = plus_mean_film.merge(mean_user, on=['userId'])
plus_mean_film_and_user.head()


,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,documentary_x,drama_x,fantasy_x,filmnoir,...,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel,rating_y,movieId_y,rating,timestamp
0,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.440594,29364.931765,2.621765,1.187732e+09
1,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.838710,29364.931765,2.621765,1.187732e+09
2,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.547619,29364.931765,2.621765,1.187732e+09
3,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.500000,29364.931765,2.621765,1.187732e+09
4,0.0,0.0,0.52611,0.0,0.263753,0.0,0.0,0.0,0.476326,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.738095,29364.931765,2.621765,1.187732e+09


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(plus_mean_film_and_user.drop(['rating_y'], axis=1), plus_mean_film_and_user['rating_y'], test_size=0.3, random_state=21)

clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_squared_error(y_train, clf.predict(X_train))))
# точность упала на 0.01 балла!

0.48261715594375854
0.43023440655428324


In [ ]:
plus_mean_film = m_dg.merge(mean_user, on=['userId'])
plus_mean_film.head()

,action_x,adventure_x,animation_x,children_x,comedy_x,crime_x,documentary_x,drama_x,fantasy_x,filmnoir,...,willfarell,witty,workneedisaymore,worldwarii,wrongfulimprisonment,zombies,zooeydeschanel,movieId_y,rating_y,timestamp
0,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29364.931765,2.621765,1.187732e+09
1,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29364.931765,2.621765,1.187732e+09
2,0.0,0.0,0.00000,0.0,0.586999,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29364.931765,2.621765,1.187732e+09
3,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29364.931765,2.621765,1.187732e+09
4,0.0,0.0,0.52611,0.0,0.263753,0.0,0.0,0.0,0.476326,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29364.931765,2.621765,1.187732e+09


In [ ]:
!pip install deepctr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.8.0
    Uninstalling h5py-3.8.0:
      Successfully uninstalled h5py-3.8.0


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(plus_mean_film.drop(['rating_y'], axis=1), plus_mean_film['rating_y'], test_size=0.3, random_state=21)

clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(np.sqrt(mean_squared_error(y_train, clf.predict(X_train))))
# точность больше на 0.07 бала, если прибавлять столбец ни среднего по фильму, а среднего по пользователю!

# ИТОГО ЭТО ЛУЧШИЙ ВАРИНАТ, которого смогла достичь регрессией: 
# к TF-IDF на тегах и жанрах добавление средней оценки пользователя

0.4021898295629839
0.3728928435223492
